In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import Adam,SGD

from sklearn.preprocessing import LabelEncoder

# from imutils import paths
import numpy as np
import argparse
import cv2
import os
import collections

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
%matplotlib inline
from xgboost import XGBClassifier

# Other Libraries
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.filterwarnings("ignore")


df = pd.read_csv('../input/yelp-reviews-dataset/yelp.csv')
df.head()

# Now we will convert target variables or we will shorten the number of classes 
namely:

0(low) class represents rating from 1-2

1(medium) class represents rating from 3

2(high) class represents rating from 4-5


In [ ]:
x = df['text'].values
df['stars']=df['stars'].replace(to_replace=[1,2],value=0)
df['stars']=df['stars'].replace(to_replace=[4,5],value=2)
df['stars']=df['stars'].replace(to_replace=[3],value=1)
y = df['stars'].values
print(x)
print(y)


# split the dataset

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=101)

# data preprocessing
# convert the sentiments to vectors 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(x_train)
x_train=vec.transform(x_train)
x_test=vec.transform(x_test)
x_train.size

# applying pca (truncatedSVD)

Principal Component Analysis (PCA) is a method that uses simple matrix operations from linear algebra and statistics to calculate a projection of the original data into the same number or fewer dimensions.

We will use TruncatedSVD

In [ ]:
train_x,test_x=x_train,x_test

pca = TruncatedSVD(n_components=3)
pca.fit(x_train)
x_train = pca.transform(x_train)
pca = TruncatedSVD(n_components=3)
pca.fit(x_test)
x_test = pca.transform(x_test)

# inference with pca

In [ ]:

model = Sequential()
model.add(Dense(36, input_dim=x_train.shape[1], activation='sigmoid'))
model.add(Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(216, activation = "sigmoid"))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(Dense(36, activation = "sigmoid"))
model.add(Dense(4, activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.1),metrics=['accuracy'])

results = model.fit(
 x_train, y_train,
 epochs= 100,
 batch_size = 500,
 validation_data = (x_test, y_test)
)

# inference without pca

In [ ]:

# without pca
model = Sequential()
model.add(Dense(36, input_dim=train_x.shape[1], activation='sigmoid'))
model.add(Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(216, activation = "sigmoid"))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(Dense(36, activation = "sigmoid"))
model.add(Dense(4, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.01),metrics=['accuracy'])

results = model.fit(
 train_x, y_train,
 epochs= 100,
 batch_size = 500,
 validation_data = (test_x, y_test)
)

**though inference without PCA gives better accuracy but the model is overfitting**

# Results  aka predicted values

In [ ]:
predictions=model.predict_classes(test_x)

In [ ]:
res=pd.DataFrame({'Rating':predictions})
res

In [ ]:
res.to_csv('Predictions.csv',index=False)